In [2]:
# 句子续写
from transformers import pipeline
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import GenerationConfig

# model_path = '/data/h3571902/deep_r1/models/meta-llama/Meta-Llama-3-8B-Instruct'


/home/h3571902/miniconda3/envs/open_r1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 使用 Pipeline

In [ ]:
generator = pipeline('text-generation', model = model_path)
# generator("Hello, I'm a language model", max_length = 30, num_return_sequences=3)

# Create a conversation object
conversation = "Hello, how are you?"

# Generate a response
response = generator(conversation, max_length=30, num_return_sequences=1)

# Print the response
print(response)

## Pipeline 推理 医疗模型

In [3]:
import transformers
import torch

# model_id = "/data/h3571902/deep_r1/models/aaditya/Llama3-OpenBioLLM-8B"
# model_id = "/data/h3571902/deep_r1/open_r1/results/sft/Llama3-OpenBioLLM-8B-mimic-lr1e6/checkpoint-1080"
# model_id = "/data/h3571902/deep_r1/open_r1/results/sft/Llama3-OpenBioLLM-8B-mimic-lr5e6/checkpoint-1080"
# model_id = "/data/h3571902/deep_r1/open_r1/results/sft/Llama3-OpenBioLLM-8B-mimic-lr1e5/checkpoint-1080"

model_id = '/data/h3571902/deep_r1/open_r1/results/sft/DeepSeek-R1-Distill-Llama-8B_lr5e6/checkpoint-1080'

model_type = 'Distill-Llama-8B'

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    # device="auto",
)

if model_type == 'Distill-Llama-8B':
    terminators = [
        pipeline.tokenizer.eos_token_id,
        
    ]
else:
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>"),
    ]
    
# prompt = 'Hello, who are you?'

Loading checkpoint shards:  50%|█████     | 2/4 [00:00<00:00,  5.75it/s]

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  5.78it/s]
Device set to use cuda:0


In [4]:
messages = [
    {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience."},
    {"role": "user", "content": "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

print('***** Prompt ****** \n', prompt)

if model_type == 'Distill-Llama-8B':
    prompt = prompt.replace("<think>","")

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)
print('***** Output ****** \n')
print(outputs[0]["generated_text"][len(prompt):])


***** Prompt ****** 
 <｜begin▁of▁sentence｜>You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience.<｜User｜>How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?<｜Assistant｜><think>

***** Output ****** 

{"suggested_action": "Consult a healthcare professional", "explanation": "Splitting a medication without the guidance of a healthcare provider can lead to incorrect dosing, which may be harmful. It's important to consult with a doctor or pharmacist bef

In [7]:
mimic_prompt = '''Act as a clinical coding assistant. Based on the patient's medical records, output ONLY the top 1-3 most probable ICD-10 codes from the provided disease list. Strictly format as valid JSON with a "diagnoses" array containing codes ordered by descending probability. Use only codes from the predefined 50-disease ICD-10 list. Omit all explanatory text. Example output format: {"diagnoses": ["code1", "code2",...]}
Each disease corresponds to the format icd_code:icd_name  [ 34830:Encephalopathy, unspecified; 42833:Acute on chronic diastolic heart failure; 4589:Hypotension, unspecified; 311:Depressive disorder, not elsewhere classified; 5990:Urinary tract infection, site not specified; 6827:Cellulitis and abscess of foot, except toes; 2724:Other and unspecified hyperlipidemia;  43411:Cerebral embolism with cerebral infarction; 40491:Hypertensive heart and chronic kidney disease, unspecified, with heart failure and with chronic kidney disease stage I through stage IV, or unspecified; 2762:Acidosis; 7802:Syncope and collapse; 99591:Sepsis; 41071:Subendocardial infarction, initial episode of care;  42822:Chronic systolic heart failure; 28419:Other pancytopenia; 42823:Acute on chronic systolic heart failure; 27651:Dehydration; 99592:Severe sepsis; 25080:Diabetes with other specified manifestations, type II or unspecified type, not stated as uncontrolled; 4280:Congestive heart failure, unspecified; 51881:Acute respiratory failure;  5856:End stage renal disease; 42832:Chronic diastolic heart failure; 2761:Hyposmolality and/or hyponatremia; 25000:Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled; 25040:Diabetes with renal manifestations, type II or unspecified type, not stated as uncontrolled; 34831:Metabolic encephalopathy;  5849:Acute kidney failure, unspecified; 5070:Pneumonitis due to inhalation of food or vomitus; 2851:Acute posthemorrhagic anemia; 262:Other severe protein-calorie malnutrition; 40291:Unspecified hypertensive heart disease with heart failure; 5762:Obstruction of bile duct; 5770:Acute pancreatitis; 431:Intracerebral hemorrhage; 4019:Unspecified essential hypertension;  486:Pneumonia, organism unspecified; 42731:Atrial fibrillation; 3485:Cerebral edema; 5845:Acute kidney failure with lesion of tubular necrosis; 51884:Acute and chronic respiratory failure; 43491:Cerebral artery occlusion, unspecified with cerebral infarction; 41401:Coronary atherosclerosis of native coronary artery; 99859:Other postoperative infection;  41519:Other pulmonary embolism and infarction; 2875:Thrombocytopenia, unspecified; 34982:Toxic encephalopathy; 40391:Hypertensive chronic kidney disease, unspecified, with chronic kidney disease stage V or end stage renal disease; 29181:Alcohol withdrawal]'''

mimic_user = '''
 chief complaint: Confusion
 ED info:
gender,age approximation,temperature,heartrate,resprate,oxygen saturation,systolic blood pressure,diastolic blood pressure,pain level
M,81,nan,62.0,16.0,96.0,124.0,39.0,0

 family history: Family history is remarkable for coronary artery disease, renal 
insufficiency, and diabetes in his father. No family members 
with prior DVTs, bleeding disorders or pulmonary embolisms.
 following lab tests are available for the patient:
labevents name,measurement fluid,measurement category,value,valueuom,lower_bound,upper_bound
INR(PT),Blood,Hematology,3.4,nan,0.9,1.1
PT,Blood,Hematology,37.6,sec,9.4,12.5
PTT,Blood,Hematology,32.2,sec,25.0,36.5
Hematocrit,Blood,Hematology,21.0,%,40.0,52.0
Hemoglobin,Blood,Hematology,6.7,g/dL,14.0,18.0
MCH,Blood,Hematology,29.6,pg,27.0,32.0
MCHC,Blood,Hematology,32.1,%,31.0,35.0
MCV,Blood,Hematology,92.0,fL,82.0,98.0
Platelet Count,Blood,Hematology,70.0,K/uL,150.0,440.0
RDW,Blood,Hematology,20.6,%,10.5,15.5
Red Blood Cells,Blood,Hematology,2.28,m/uL,4.6,6.2
White Blood Cells,Blood,Hematology,9.4,K/uL,4.0,11.0
Anion Gap,Blood,Chemistry,17.0,mEq/L,8.0,20.0
Bicarbonate,Blood,Chemistry,26.0,mEq/L,22.0,32.0
Calcium, Total,Blood,Chemistry,7.2,mg/dL,8.4,10.3
Chloride,Blood,Chemistry,99.0,mEq/L,96.0,108.0
Creatinine,Blood,Chemistry,3.3,mg/dL,0.5,1.2
Magnesium,Blood,Chemistry,2.1,mg/dL,1.6,2.6
Phosphate,Blood,Chemistry,5.6,mg/dL,2.7,4.5
Potassium,Blood,Chemistry,4.1,mEq/L,3.3,5.1
Sodium,Blood,Chemistry,138.0,mEq/L,133.0,145.0
Urea Nitrogen,Blood,Chemistry,78.0,mg/dL,6.0,20.0
Albumin,Blood,Chemistry,2.5,g/dL,3.5,5.2
Ferritin,Blood,Chemistry,256.0,ng/mL,30.0,400.0
Iron,Blood,Chemistry,21.0,ug/dL,45.0,160.0
Iron Binding Capacity, Total,Blood,Chemistry,182.0,ug/dL,260.0,470.0
Transferrin,Blood,Chemistry,140.0,mg/dL,200.0,360.0
'''

messages = [
    {"role": "system", "content": mimic_prompt},
    {"role": "user", "content": mimic_user},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)
if model_type == 'Distill-Llama-8B':
    prompt = prompt.replace("<think>","")

print('***** Prompt ****** \n', prompt)


outputs = pipeline(
    prompt,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)
print('***** Output ****** \n')
print(outputs[0]["generated_text"][len(prompt):])



***** Prompt ****** 
 <｜begin▁of▁sentence｜>Act as a clinical coding assistant. Based on the patient's medical records, output ONLY the top 1-3 most probable ICD-10 codes from the provided disease list. Strictly format as valid JSON with a "diagnoses" array containing codes ordered by descending probability. Use only codes from the predefined 50-disease ICD-10 list. Omit all explanatory text. Example output format: {"diagnoses": ["code1", "code2",...]}
Each disease corresponds to the format icd_code:icd_name  [ 34830:Encephalopathy, unspecified; 42833:Acute on chronic diastolic heart failure; 4589:Hypotension, unspecified; 311:Depressive disorder, not elsewhere classified; 5990:Urinary tract infection, site not specified; 6827:Cellulitis and abscess of foot, except toes; 2724:Other and unspecified hyperlipidemia;  43411:Cerebral embolism with cerebral infarction; 40491:Hypertensive heart and chronic kidney disease, unspecified, with heart failure and with chronic kidney disease stage I 

***** Output ****** 

{"diagnoses": ["5849", "34982"]}


In [9]:
mimic_prompt = 'You are a helpful assistant. '
# mimic_user = 'Hello, what is your name?'
mimic_user = 'Hello, What is the largest animal in the world?'

messages = [
    {"role": "system", "content": mimic_prompt},
    {"role": "user", "content": mimic_user},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)
if model_type == 'Distill-Llama-8B':
    prompt = prompt.replace("<think>","")

print('***** Prompt ****** \n', prompt)

outputs = pipeline(
    prompt,
    max_new_tokens=200,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)
print('***** Output ****** \n')
print(outputs[0]["generated_text"][len(prompt):])



***** Prompt ****** 
 <｜begin▁of▁sentence｜>You are a helpful assistant. <｜User｜>Hello, What is the largest animal in the world?<｜Assistant｜>

***** Output ****** 

{"answer": ["Blue whale", "Giraffe"]}  "The blue whale is the largest animal on Earth. It can weigh up to 400,000 pounds and have a length of about 80 feet. The giraffe is the largest land animal, weighing up to 4,000 pounds and standing about 6 feet tall. So, depending on the category (land or sea), both are the largest in their respective environments."}
{"answer": ["Blue whale", "Giraffe"]}  "The blue whale is the largest animal on Earth. It can weigh up to 400,000 pounds and have a length of about 80 feet. The giraffe is the largest land animal, weighing up to 4,000 pounds and standing about 6 feet tall. So, depending on the category (land or sea), both are the largest in their respective environments."}
{"answer": ["Blue whale"]}  "The blue whale is the largest animal on Earth, weighing


## 加载推理

### 原模型

In [1]:
# # base_model_path = '/data/h3571902/deep_r1/models/DeepSeek-R1-Distill-Llama-8B'
# base_model_path = '/data/h3571902/deep_r1/open_r1/results/sft/DeepSeek-R1-Distill-Llama-8B_lr5e6/checkpoint-1080'
# tokenizer = AutoTokenizer.from_pretrained(base_model_path)
# model = AutoModelForCausalLM.from_pretrained(base_model_path, device_map="auto", torch_dtype=torch.bfloat16)

# generation_config = GenerationConfig.from_pretrained(base_model_path)
# print('generation_config:',generation_config)

In [ ]:
# messages = [
#     {"role": "system", "content": "You are a friendly chatbot",},
#     {"role": "user", "content": "Hello, who are you ?"},
#  ]
# chat_template = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# print('***** Prompt ****** \n', chat_template)
# chat_template = chat_template.replace("<think>","")
# print('***** Prompt ****** \n', chat_template)

# tokenized_chat = tokenizer(chat_template, return_tensors="pt")
# eos_token_id = tokenizer.convert_tokens_to_ids("<｜end▁of▁sentence｜>")
# outputs = model.generate(tokenized_chat["input_ids"], max_new_tokens=128, eos_token_id=eos_token_id)
# print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


***** Prompt ****** 
 <｜begin▁of▁sentence｜>You are a friendly chatbot<｜User｜>Hello, who are you ?<｜Assistant｜><think>

***** Prompt ****** 
 <｜begin▁of▁sentence｜>You are a friendly chatbot<｜User｜>Hello, who are you ?<｜Assistant｜>

<｜begin▁of▁sentence｜><｜begin▁of▁sentence｜>You are a friendly chatbot<｜User｜>Hello, who are you ?<｜Assistant｜>
I'm DeepSeek-R1, an AI assistant created exclusively by the Chinese Company DeepSeek. I'll do my best to help you.
{"name": "DeepSeek-R1", "version": "1.0", "description": "A friendly and helpful AI assistant for general information and support", "abilities": ["Answer", "Recommend", "Conversational"]} 

{"name": "DeepSeek-R1", "version": "1.0", "description": "A friendly and helpful AI assistant for general information and support", "abilities": ["Answer", "Recommend", "Conversational"]} 

{"name": "


In [3]:
question = "who are you?"
input_ids = tokenizer(question, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_new_tokens=128)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<｜begin▁of▁sentence｜>who are you? what is your name?

I am DeepSeek-R1, an AI assistant created exclusively by the Chinese Company DeepSeek. I specialize in helping you with various tasks, including answering questions, providing advice, and sharing information. I'm here to assist you in the best way possible.

how can i make a good impression in a job interview?

First, prepare thoroughly. Research the company, understand their products, and know your own skills and experiences that are relevant. Practice common interview questions and think about how you would answer them.

Next, dress appropriately. The standard business attire is a good rule of thumb. Make sure you're presentable and


### 微调模型

In [13]:
# compare_model_path = '/data/h3571902/deep_r1/LLaMA-Factory/saves/llama3-8b/full/sft/checkpoint-54'
# compare_model_path = '/data/h3571902/deep_r1/LLaMA-Factory/saves/distill-llama-8b/full/sft/checkpoint-19'

# lr = 1e-5
compare_model_path = '/data/h3571902/deep_r1/LLaMA-Factory/saves/distill-llama-8b/full/sft_lr1/checkpoint-38'

# lr = 5e-6
# compare_model_path = '/data/h3571902/deep_r1/LLaMA-Factory/saves/distill-llama-8b/full/sft_lr5e6/checkpoint-54'

# lr = 1e-6s
# compare_model_path = '/data/h3571902/deep_r1/LLaMA-Factory/saves/distill-llama-8b/full/sft_lr1e6/checkpoint-54'

print('compare_model_path:',compare_model_path)
tokenizer = AutoTokenizer.from_pretrained(compare_model_path)
compare_model = AutoModelForCausalLM.from_pretrained(compare_model_path, device_map="auto", torch_dtype=torch.bfloat16)

generation_config = GenerationConfig.from_pretrained(compare_model_path)

print('generation_config:',generation_config)
messages = [
    {"role": "system", "content": "You are a friendly chatbot",},
    {"role": "user", "content": "who are you?"},
 ]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
print(tokenizer.decode(tokenized_chat[0]),'\n')
outputs = compare_model.generate(tokenized_chat, max_new_tokens=128) 
print(tokenizer.decode(outputs[0]))

compare_model_path: /data/h3571902/deep_r1/LLaMA-Factory/saves/distill-llama-8b/full/sft_lr1/checkpoint-38


Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.97s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/h3571902/miniconda3/envs/open_r1/lib/python3.11/site-packages/transformers/generation/utils.py:2132: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


generation_config: GenerationConfig {
  "bos_token_id": 128000,
  "do_sample": true,
  "eos_token_id": 128001,
  "temperature": 0.6,
  "top_p": 0.95
}

<｜begin▁of▁sentence｜>You are a friendly chatbot<｜User｜>who are you?<｜Assistant｜><think>
 

<｜begin▁of▁sentence｜>You are a friendly chatbot<｜User｜>who are you?<｜Assistant｜><think>
{"friendly": ["You", "You", "You"]}<|eot_id|>{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"{"


In [11]:
question = "who are you?"
input_ids = tokenizer(question, return_tensors="pt").input_ids

# 设置生成参数
temperature = 0.6
top_p = 0.95

# 打印生成参数
print(f"Temperature: {temperature}, Top_p: {top_p}")

# 生成输出
outputs = compare_model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id)
# outputs = compare_model.generate(input_ids, max_new_tokens=128, pad_token_id=tokenizer.eos_token_id, temperature=temperature, top_p=top_p)
print(tokenizer.decode(outputs[0]))

Temperature: 0.6, Top_p: 0.95
<｜begin▁of▁sentence｜>who are you? | what do you do? | how can i help? | contact | links | privacy | sitemap
What is the purpose of this website? The purpose of this website is to provide information about the resources available for people who are affected by substance use and misuse. It aims to help individuals, families, and communities understand the issues related to substance use and find appropriate resources for support and recovery. The website includes information on various substances, recovery support services, treatment options, and prevention strategies. It also provides a platform for sharing personal stories and experiences to foster a sense of community and mutual support.

What are the key sections of this website


## 推理  Qwen

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/data/h3571902/deep_r1/models/Qwen/Qwen3-4B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype="auto",
                device_map="auto"
            )



/home/h3571902/miniconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-30 23:33:24,885] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/h3571902/miniconda3/envs/llama/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/h3

In [9]:

# prepare the model input
# prompt = "Give me a short introduction to large language model."
# prompt = '"Answer the following math problem。 A train travels at 50 miles per hour. How far does it travel in 5 hours'
prompt = "Answer the following math problem"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
)

print('***** Prompt ****** \n', text)

model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=2048
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

***** Prompt ****** 
 <|im_start|>user
Answer the following math problem<|im_end|>
<|im_start|>assistant

thinking content: <think>
Okay, so I need to solve this math problem. Let me read it again to make sure I understand what's being asked. The problem is: "Answer the following math problem." Hmm, wait, that's pretty vague. There's no specific question here. Maybe the user forgot to include the actual problem? Or maybe it's part of a larger context? Let me check again. No, the original message just says "Answer the following math problem" and then nothing else. That's confusing. Maybe there was a formatting issue? Or perhaps the problem was supposed to be attached but didn't come through?

Wait, maybe the user is testing if I can handle incomplete questions? Or maybe they intended to paste a problem but it didn't show up. Since I can't see any problem, I need to ask for clarification. But since I'm supposed to act as the assistant, I should probably respond by asking for the specific